In this demo, we demonstrate how to use the MaxEnt_Reg script. We will be going over how to do the following:

1. Reading in the MaxEnt data and constraint information
2. Running the model using different gradient descent techniques (without regularization)
3. How to add regularization (target prior ```TGTPrior``` vs. difference prior ```DIFPrior```)

In [3]:
## Good old NumPy
import numpy as np

## Specialized classes
from utils.MaxEnt import MaxEnt
from utils.Regularization import TGTPrior, DIFPrior

## Utility function for reading in constraint and data information
from utils.OTSoft_file_reader import get_info

For this demonstration, we will be looking at the Hayes Pseudo-Korean data

In [5]:
## Read in data and constraint information
constraint_names, underlying_forms, candidates, violations, observed_probs = get_info("toy_datasets/HayesPseudoKorean-RichBase.txt")

## Initialize a MaxEnt object
me = MaxEnt(constraint_names)

## Perform learning without regularization
me.SGD_learn(violations = violations, observed_prob = observed_probs)

['Ident (asp)' 'Ident (voice)' 'Ident (asp)/_V' 'Ident (voice)/_V'
 '*[+v][-v][+v]' '*dh' '*[-son/+voice]' '*aspiration']
[array([1., 0., 0., 0.]), array([1., 0., 0., 0.]), array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([1., 0., 0., 0.]), array([1., 0., 0., 0.]), array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([1., 0., 0., 0.]), array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

Traceback (most recent call last):
  File "/Users/christopheryang/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/wr/nbvfw0zs3jq6h2mh_38k6dxh0000gn/T/ipykernel_4391/2047592465.py", line 10, in <module>
    me.SGD_learn(violations = violations, observed_prob = observed_probs)
  File "/Users/christopheryang/Documents/projects/MaxEnt_Reg/utils/MaxEnt.py", line 148, in SGD_learn
  File "/Users/christopheryang/Documents/projects/MaxEnt_Reg/utils/MaxEnt.py", line 57, in get_weighted_winner_violations
    rst.append(weighted_tab[~np.all(weighted_tab == 0, axis=1)])
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<__array_function__ internals>", line 200, in all
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py", line 2515, in all
    return _wrapreduction(a, np.logical_and, 'all'